In [ ]:
#DATA CLEANING

In [ ]:
# Standard libraries 
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import re

# Importing data
df_raw = pd.read_csv('original_dataset.csv')

# Creating a copy of the raw data to mantain the original version just in case
df = df_raw.copy()

In [ ]:
#Salva file modificato
df.to_csv('cleaned_dataset.csv')

In [ ]:
# Dataset visualisation
df.head(5)

In [ ]:
# Columns types and non-null count
df.info()

In [ ]:
# Definisci una funzione per estrarre l'anno da una stringa usando regex
def estrai_anno_da_stringa(testo):
    year = re.findall(r'\b\d{4}\b', testo)  # Trova tutti i pattern YYYY nel testo
    if year:
        return int(year[0])  # Prendi il primo anno trovato come intero
    else:
        return None

# Applica la funzione al campo 'title' per estrarre l'anno
df['year'] = df['title'].apply(estrai_anno_da_stringa)

# Gestisci i valori NaN e infiniti sostituendoli con 0
df['year'].fillna(0, inplace=True)

# Trasforma la colonna "anni" in valori interi
df['year'] = df['year'].astype(int)

In [ ]:
# Funzione per copiare il valore da "title" a "designation se "designation" è nullo
def estrai_designation_da_title(row):
    if pd.isna(row['designation']):
        # Utilizza un'espressione regolare per estrarre "designation"
        result = re.search(r'\d{4}\s(.+?)\s\(', row['title'])
        if result:
            row['designation'] = result.group(1)
        else:
            # Se l'estrazione da "title" è nullo, copia il contenuto di "variety"
            row['designation'] = row['variety']
    return row

# Applica la funzione a ciascuna riga del DataFrame
df = df.apply(estrai_designation_da_title, axis=1)

In [ ]:
# Dropping unnecessary columns
df.drop(['Unnamed: 0', 'province', 'region_1', 'region_2', 
         'title', 'taster_name', 'taster_twitter_handle'], 
        axis=1, inplace=True)

In [ ]:
# Definisci l'ordine desiderato delle colonne
nuovo_ordine_colonne = ['country', 'description', 'winery', 'variety', 
                        'designation', 'year', 'points', 'price']

# Seleziona le colonne nel nuovo ordine
df = df[nuovo_ordine_colonne]

In [ ]:
# Calcola il numero di valori nulli per ogni colonna
null_counts = df.isnull().sum()

# Calcola il totale di valori in ogni colonna
total_counts = df.shape[0]

# Calcola la percentuale dei valori nulli
null_percentage = (null_counts / total_counts * 100).round(2).astype(str) + '%'

# Crea un nuovo DataFrame con i risultati
null_info = pd.DataFrame({'Null Count': null_counts, 'Percentage': null_percentage})

print(null_info)

In [ ]:
# Ricerca dei valori mancanti di designation
filtered_df = df[df['designation'].isnull()]
filtered_df

In [ ]:
# Elimina le righe con almeno un valore nullo di price e designation
df.dropna(subset=['price', 'designation'], inplace=True)

In [ ]:
#DATA ANALYSIS E VISUALISATION

In [ ]:
#Analisi statistica del campo points
df['points'].describe()

In [ ]:
# Points Distribution
plt.hist(df["points"], bins=20, density=True, rwidth=0.8)  # Puoi regolare il valore di rwidth
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Imposta l'asse sotto agli altri elementi

# Aggiungi le etichette agli assi
plt.xlabel('Points')
plt.ylabel('Percentage of Density')

plt.show()

In [ ]:
#Analisi statistica del campo price
df['price'].describe()

In [ ]:
# Price Distribution
plt.hist(df["price"], bins=50, density=True, rwidth=0.8)  # Puoi regolare il valore di rwidth
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Imposta l'asse sotto agli altri elementi

# Aggiungi le etichette agli assi
plt.xlabel('Price')
plt.ylabel('Percentage of Density')

plt.show()

In [ ]:
# Calcola il 90° percentile dei prezzi
price_90th_percentile = df['price'].quantile(0.9)

# Filtra il DataFrame includendo 
# solo i prezzi entro il 90° percentile
filtered_df = df[df['price'] <= price_90th_percentile]

filtered_df['price'].describe()

In [ ]:
# Price Distribution considering 90th percentile
plt.hist(df["price"], range = (0,65), bins=20, density=True, rwidth=0.8)  # Puoi regolare il valore di rwidth
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Impostazione l'asse sotto agli altri elementi

# Aggiungi le etichette agli assi
plt.xlabel('Price')
plt.ylabel('Percentage of Density')

plt.show()

In [ ]:
# Classifica dei vini per nazione

# Calcola la frequenza delle varie nazioni
country_counts = df['country'].value_counts()

# Seleziona solo le prime 10 nazioni
top_countries = country_counts.head(10)

# Calcola le percentuali
percentages = (top_countries / top_countries.sum()) * 100

# Plotta il diagramma a barre con colori diversi
plt.figure(figsize=(10, 6))
bars = plt.bar(top_countries.index, percentages, color=plt.cm.Paired.colors)

# Mostra il nome della nazione in orizzontale
plt.xticks(rotation=45, ha='right')

# Aggiungi la griglia come sfondo
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Impostazione l'asse sotto agli altri elementi

# Aggiungi le percentuali sopra ogni barra con il simbolo %
for bar, percent in zip(bars, percentages):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5,
             f'{percent:.2f}%', ha='center', va='bottom')

# Aggiungi il simbolo % all'asse delle ordinate
plt.gca().set_yticklabels([f'{int(tick)}%' for tick in plt.gca().get_yticks()])

plt.title('Top 10 Nazioni per Frequenza (Percentuali sul Totale)')
plt.xlabel('Nazione')
plt.ylabel('Percentuale sul Totale')

plt.show()

In [ ]:
# Classifica delle varietà vinicole per nazione

# Calcola il numero di varietà uniche per ciascun paese
unique_varieties_by_country = df.groupby('country')['variety'].nunique().sort_values(ascending=False)

# Seleziona i primi 10 paesi
top_10_countries = unique_varieties_by_country.head(10)

# Calcola le percentuali
total_varieties = top_10_countries.sum()
percentages = (top_10_countries / total_varieties) * 100

# Plotta il diagramma a barre con colori diversi
plt.figure(figsize=(10, 6))
bars = plt.bar(top_10_countries.index, percentages, color=plt.cm.Paired.colors)

# Aggiungi etichette percentuali sopra ogni barra
for bar, percentage in zip(bars, percentages):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f'{percentage:.2f}%', ha='center', va='bottom')
    
# Aggiungi la griglia come sfondo
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Imposta l'asse sotto agli altri elementi

plt.xlabel('Country')
plt.ylabel('Number of Unique Varieties')
plt.title('Top 10 Countries by Number of Unique Varieties')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Mostra il grafico
plt.show()

In [ ]:
# Classifica dei vini in base alla media punteggi

# Conta il numero di record per ogni paese
country_counts = df['country'].value_counts()

# Seleziona solo i paesi con almeno 1000 record
valid_countries = country_counts[country_counts >= 1000].index

# Filtra il DataFrame solo per i paesi validi
df_filtered = df[df['country'].isin(valid_countries)]

# Calcola la media di punteggi per ogni paese
mean_points_by_country = df_filtered.groupby('country')['points'].mean()

# Arrotonda la media a due decimali
mean_points_by_country = mean_points_by_country.round(2)

# Ordina i paesi in base alla media di punteggi in ordine decrescente
ranked_countries = mean_points_by_country.sort_values(ascending=False)

# Crea una tabella con la classifica
ranked_table = pd.DataFrame({'Nazione': ranked_countries.index, 'Media Punteggi': ranked_countries.values})

ranked_table

In [ ]:
# Classifica dei vini in base alla media prezzi

# Conta il numero di record per ogni paese
country_counts = df['country'].value_counts()

# Seleziona solo i paesi con almeno 1000 record
valid_countries = country_counts[country_counts >= 1000].index

# Filtra il DataFrame solo per i paesi validi
df_filtered = df[df['country'].isin(valid_countries)]

# Calcola la media dei prezzi per ogni paese
mean_price_by_country = df_filtered.groupby('country')['price'].mean()

# Arrotonda la media a due decimali
mean_price_by_country = mean_price_by_country.round(2)

# Ordina i paesi in base alla media dei prezzi in ordine decrescente
ranked_countries = mean_price_by_country.sort_values(ascending=False)

# Crea una tabella con la classifica
ranked_table = pd.DataFrame({'Nazione': ranked_countries.index, 'Media Prezzi': ranked_countries.values})

ranked_table

In [ ]:
# Correlazione prezzo-qualità

# Filtra il DataFrame per includere solo i vini con prezzi fino a 65 € (90mo percentile)
filtered_df = df[df['price'] <= 200]

# Creazione del grafico di correlazione
plt.figure(figsize=(10, 8))
sns.scatterplot(x='points', y='price', data=filtered_df, color='purple', alpha=0.6)
plt.xlabel('Points')
plt.ylabel('Price (Euro)')
plt.title('Correlation between Wine Prices and Points')
plt.tight_layout()

# Mostra il grafico
plt.show()

In [ ]:
# Calcola la matrice di correlazione prezzo-qualità

correlation_matrix = df[['price', 'points']].corr()

# Creazione della heatmap della matrice di correlazione con seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix: Price vs Points')
plt.show()

In [ ]:
# Filtra i vini con anno maggiore o uguale a 1472
filtered_wines = df[df['year'] >= 1472]

# Ordina la tabella in modo crescente rispetto al campo 'year' e seleziona le prime 10 righe
result_table = filtered_wines.sort_values(by='year').head(10)

# Seleziona solo i campi desiderati
selected_columns = ['year', 'variety', 'price']

# Mostra la tabella risultante
print(result_table[selected_columns])


In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(df['year'], bins=30, kde=True, color='skyblue')
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Distribution of Wine Years')
plt.show()


In [ ]:
filtered_years = df[(df['year'] >= 1980) & (df['year'] <= 2023)]

plt.figure(figsize=(12, 8))
sns.histplot(filtered_years['year'], bins=30, kde=True, color='skyblue')
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Distribution of Wine Years (1472-2023)')
plt.show()


In [ ]:
# Filtra il DataFrame per includere solo i vini con prezzo massimo di 1000 € e anni tra 1995 e 2023
filtered_wines = df[(df['price'] <= 1000) & (df['year'] >= 1995) & (df['year'] <= 2023)]

plt.figure(figsize=(12, 8))
sns.scatterplot(x='year', y='price', data=filtered_wines, color='skyblue', alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Price')
plt.title('Scatter Plot of Wine Years and Prices (1995-2023, Price <= 1000 €)')
plt.show()

In [ ]:
# Filtra il DataFrame per includere solo i vini con prezzo massimo di 1000 €
filtered_wines = df[df['price'] <= 1000]

plt.figure(figsize=(12, 8))
sns.scatterplot(x='year', y='price', data=filtered_wines, color='skyblue', alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Price')
plt.title('Scatter Plot of Wine Years and Prices (Price <= 1000 €)')
plt.show()


In [ ]:
filtered_wines = df[(df['price'] <= 1000) & (df['year'] >= 1995) & (df['year'] <= 2023)]

plt.figure(figsize=(12, 8))
sns.heatmap(filtered_years[['price', 'year']].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap between Price and Year')
plt.show()


In [ ]:
# Classifica dei vini più costosi

# Ordina il DataFrame in base al prezzo in modo decrescente
df_sorted = df.sort_values(by='price', ascending=False)

# Estrai i primi 10 vini più costosi
top_10_wines = df_sorted.head(10)

# Seleziona solo le colonne desiderate e azzerare gli indici
selected_columns = ['country', 'variety', 'price']
top_10_wines_selected = top_10_wines[selected_columns].reset_index(drop=True)

top_10_wines_selected

In [ ]:
# Classifica delle eccellenze

# Filtra il DataFrame per vini con punteggio tra 98 e 100
excellent_wines = df[(df['points'] >= 98) & (df['points'] <= 100)]

# Conta il numero di vini eccellenti per ogni paese
excellent_counts = excellent_wines['country'].value_counts()

# Seleziona solo i paesi con almeno un vino eccellente
valid_countries = excellent_counts[excellent_counts > 0].index

# Ordina i paesi in base al totale di vini eccellenti in ordine decrescente
ranked_countries = excellent_counts.loc[valid_countries].sort_values(ascending=False)

# Plotta il grafico a barre
plt.figure(figsize=(10, 6))
bars = plt.bar(ranked_countries.index, ranked_countries.values, color=plt.cm.Paired.colors)

# Mostra il totale di vini eccellenti sulla cima di ogni barra
for bar, count in zip(bars, ranked_countries.values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.1,
             f'{count}', ha='center', va='bottom')

plt.title('Classifica Paesi con più Vini Eccellenti (punteggio tra 98 e 100)')
plt.xlabel('Nazione')
plt.ylabel('Totale Vini Eccellenti')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Imposta l'asse sotto agli altri elementi

plt.show()

In [ ]:
# Classifica vini per varietà

# Calcola la frequenza delle varietà
variety_counts = df['variety'].value_counts()

# Seleziona solo le prime 10 varietà
top_varieties = variety_counts.head(10)

# Calcola le percentuali
percentages = (top_varieties / top_varieties.sum()) * 100

# Plotta il diagramma a barre con colori diversi
plt.figure(figsize=(10, 6))
bars = plt.bar(top_varieties.index, percentages, color=plt.cm.Paired.colors)

# Mostra il nome della nazione in orizzontale
plt.xticks(rotation=45, ha='right')

# Aggiungi la griglia come sfondo
plt.grid(True, alpha=0.5)
plt.gca().set_axisbelow(True)  # Imposta l'asse sotto agli altri elementi

# Aggiungi le percentuali sopra ogni barra con il simbolo %
for bar, percent in zip(bars, percentages):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 2,
             f'{percent:.2f}%', ha='center', va='bottom')

# Aggiungi il simbolo % all'asse delle ordinate
plt.gca().set_yticklabels([f'{int(tick)}%' for tick in plt.gca().get_yticks()])

# Aggiungi spazio tra la barra più alta e il perimetro del grafico
plt.ylim(0, max(percentages) + 3)

plt.title('Top 10 Varietà per Frequenza (Percentuali sul Totale)')
plt.xlabel('Varietà')
plt.ylabel('Percentuale sul Totale')

plt.show()

In [ ]:
# Classifica varietà del vini per Paese

#Calcola la frequenza percentuale delle varietà con i rispettivi paesi produttori
top_varieties_with_country = df.groupby(['variety', 'country']).size().reset_index(name='frequency')
total_entries = len(df)  # Calcola il totale delle righe nel DataFrame

# Calcola la frequenza percentuale
top_varieties_with_country['percentage'] = (top_varieties_with_country['frequency'] / total_entries) * 100

# Ordina per frequenza percentuale in modo decrescente e ottieni i primi 10 risultati
top_varieties_with_country = top_varieties_with_country.sort_values(by='percentage', ascending=False).head(10)

# Creazione del grafico a barre verticali con percentuali etichettate
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='variety', y='percentage', hue='country', data=top_varieties_with_country, dodge=True)
plt.xlabel('Variety')
plt.ylabel('Percentage')
plt.title('Top 10 Varieties with Country of Origin (Percentage)')

# Aggiungi etichette percentuali sopra le barre
for p in ax.patches:
    height = p.get_height()
    plt.text(p.get_x() + p.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')

plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Mostra il grafico
plt.show()

In [ ]:
#CREAZIONE DEL CATALOGO

In [ ]:
# Filtra il DataFrame per vini con prezzo fino a 100 €
filtered_wines = df[df['price'] <= 100]

# Ordina i vini in base al punteggio in ordine decrescente e prendi i primi 50
top_50_wines = filtered_wines.sort_values(by='points', ascending=False).head(50)

# Resetta l'indice in modo che parta da 1
top_50_wines.reset_index(drop=True, inplace=True)

top_50_wines

In [ ]:
#Salva file modificato
top_50_wines.to_csv('catalogue.csv')